## ANN model for finding the best parameter for measuring geometric attributes of a location for opening a new restaurant

In [ ]:
# Training data: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\training-data
# Testing data: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\valid-data
import pandas as pd

# Read data: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\training-data\trs210531_geo_mapped.csv
data = pd.read_csv('C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\prepared-data\\training-data\\trs210531_geo_mapped.csv')

# data with different 'TYPE' and 'INFO', should be split into different dataframes, as they are different types of categories
# but in this sample, we won't do that, we will just use the whole data to train the model
# features columns: data.iloc[:,7:], by weighting the features, we can guess whether this restaurant would be survived or not
# Append a new column 'survive' to the dataframe, 1 means survived, 0 means not survived
# valid data: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\valid-data\trs210531_geo_mapped.csv
# if each row of the valid data has the same 'survive' value, then the model is good; otherwise, the model is not good

# model training: data.iloc[:,7:] is the features columns which will be added as the 17 neurons to the model; use MLP model to train the model
# use 10 fold cross validation to train the model, and use the average accuracy as the final accuracy

In [19]:
# training datas: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\training-data
# valid datas: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\valid-data
import pandas as pd
import numpy as np
from sklearn import preprocessing
import math

# sigmoid function
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# make a load test data function
def load_test_data(test_data_path, weight = [0]):
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.iloc[:,7:]
    # standardize the data
    test_data_norm = preprocessing.scale(test_data)
    # if the weight is not given, then initial the weight by random for deep learning
    # weight would be range from -1 to 1, having the same length as the features columns
    if weight == [0]:
        weight = np.random.uniform(-1,1,len(test_data.columns))
    # multiply the weight to the test_data_norm, and then use sigmoid function to get the result, append the result to the test_data['survive']
    test_data['survive'] = test_data_norm.dot(weight)
    test_data['survive'] = test_data['survive'].apply(sigmoid)
    # after applying sigmoid function, the result would be range from 0 to 1, so we can use 0.5 as the threshold to determine whether the restaurant would be survived or not
    test_data['survive'] = test_data['survive'].apply(lambda x: 1 if x >= 0.5 else 0)
    return test_data, weight

# make a load valid data function
def load_training_data(training_data_path):
    training_data = pd.read_csv(training_data_path)
    training_data = training_data.iloc[:,-1]
    return training_data

def validation(test_data, training_data):
    # count how many test_data['survive'] is the same as training_data
    count = 0
    for i in range(len(test_data)):
        if test_data.iloc[i,-1] == training_data.iloc[i]:
            count += 1
    # calculate the accuracy
    accuracy = count / len(test_data)
    return accuracy

# load the first training data from folder C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\training-data
# for the first time, the weight is not given, so the weight would be initial by random
# after the first time, vertify the model by using the first valid data from folder C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\valid-data
# if there's unsatisfied accuracy, then adjust the weight, and use the adjusted weight to vertify the model again
# if the accuracy is satisfied, then use the next training data to train the model, and use the next valid data to vertify the model
# repeat the process until the last training data and valid data
# after the last valid data, we can get the final accuracy

# load the first training data
test_data, weight = load_test_data('C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\prepared-data\\training-data\\trs210531_geo_mapped.csv')
# load the first valid data
training_data = load_training_data('C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\prepared-data\\valid-data\\trs210531_geo_mapped.csv')
# vertify the model
accuracy = validation(test_data, training_data)
# if the accuracy is not satisfied, then adjust the weight, and use the adjusted weight to vertify the model again
count = 0
while count < 10:
    # adjust the weight by difference between test_data['survive'] and training_data
    # if test_data['survive'] is 1, but training_data is 0, then the weight should be added
    # if test_data['survive'] is 0, but training_data is 1, then the weight should be subtracted
    # if test_data['survive'] is the same as training_data, then the weight should be the same
    for i in range(len(test_data)):
        if test_data.iloc[i,-1] == 1 and training_data.iloc[i] == 0:
            weight += test_data.iloc[i,:-1]
        elif test_data.iloc[i,-1] == 0 and training_data.iloc[i] == 1:
            weight -= test_data.iloc[i,:-1]
    # convert weight having shape (17,) to (17,1), by the average of each row
    # use the adjusted weight to vertify the model again
    # vertify the model again
    accuracy = validation(test_data, training_data)
    print(accuracy)
    print(np.array(weight))
    count += 1
# if the accuracy is satisfied, then use the next training data to train the model, and use the next valid data to vertify the model
# repeat the process until the last training data and valid data
# after the last valid data, we can get the final accuracy

print(accuracy)

0.5222984562607204
[-4.62315679e+03 -1.04394070e+03 -4.89692788e+02 -6.53980162e+02
 -1.61969851e+03 -6.91608758e+02  9.39889944e-01 -4.99944783e+02
 -5.24404251e+02             nan             nan             nan
             nan             nan             nan             nan
             nan]
0.5222984562607204
[-9.24715679e+03 -2.08694070e+03 -9.78692788e+02 -1.30798016e+03
 -3.23969851e+03 -1.38260876e+03  9.39889944e-01 -9.99944783e+02
 -1.04840425e+03             nan             nan             nan
             nan             nan             nan             nan
             nan]
0.5222984562607204
[-1.38711568e+04 -3.12994070e+03 -1.46769279e+03 -1.96198016e+03
 -4.85969851e+03 -2.07360876e+03  9.39889944e-01 -1.49994478e+03
 -1.57240425e+03             nan             nan             nan
             nan             nan             nan             nan
             nan]
0.5222984562607204
[-1.84951568e+04 -4.17294070e+03 -1.95669279e+03 -2.61598016e+03
 -6.47969851e+03 -2.76460

In [20]:
# as per above logic, build an ANN model with 17 neurons, and 1 output layer with sigmoid function to determine which weight is the best for predicting the survive rate
# use 10 fold cross validation to train the model, and use the average accuracy as the final accuracy
# training datas: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\training-data
# valid datas: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\valid-data
import pandas as pd
import numpy as np
from sklearn import preprocessing
import math
from sklearn.model_selection import KFold
from keras.models import Sequential # intialize the ANN
from keras.layers import Dense # build the layers of ANN
from keras.layers import Dropout # dropout some neurons to prevent overfitting
from sklearn.model_selection import cross_val_score # use cross validation to train the model
from sklearn.model_selection import GridSearchCV # find the best parameters for the model
from scikeras.wrappers import KerasClassifier # wrap the keras model into scikit learn
from sklearn.model_selection import StratifiedKFold # use stratified k fold to train the model

# function to load the test data
def load_test_data(test_data_path):
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.iloc[:,7:]
    # standardize the data
    test_data_norm = preprocessing.scale(test_data)
    return test_data_norm
  
# function to load the desire output training data
def load_training_data(training_data_path):
    training_data = pd.read_csv(training_data_path)
    training_data = training_data.iloc[:,-1]
    return training_data

# sigmoid function
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# make a sequential ANN model
def build_classifier(init, neurons):
    classifier = Sequential()
    # add the input layer and the first hidden layer
    classifier.add(Dense(units = neurons, kernel_initializer= init, activation = 'relu', input_dim = neurons))
    classifier.add(Dropout(rate = 0.1))
    # add the second hidden layer
    classifier.add(Dense(units = neurons, kernel_initializer = init, activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    # add the output layer
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

# input training data iloc[:,7:], desire output training data iloc[:,-1]
# training data paths: C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\prepared-data\valid-data
# output the best parameters for the model

# open the training data folder
import os
training_data_folder = 'C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\prepared-data\\training-data'
desire_output_folder = 'C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\prepared-data\\valid-data'
training_data_list = os.listdir(training_data_folder)

# load the training data
training_data = load_test_data(training_data_folder + '\\' + training_data_list[0])
# load the desire output training data
training_data_output = load_training_data(desire_output_folder + '\\' + training_data_list[0])

# directory to save the best parameters for the model is C:\Competition\coding-with-git-repo\open-data-hackthon\becky\training-model\model-training-result
# if the directory does not exist, then create the directory
if not os.path.exists('C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\model-training-result'):
    os.makedirs('C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\model-training-result')
    
# file paths for saving the best parameters for the model
final_model_param = 'C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\model-training-result\\final_model_param.csv'
final_model_accuracy = 'C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\model-training-result\\final_model_accuracy.csv'
final_model_weight = 'C:\\Competition\\coding-with-git-repo\\open-data-hackthon\\becky\\training-model\\model-training-result\\final_model_weight.csv'

# predict the best parameters for the model
# use 10 fold cross validation to train the model, and use the average accuracy as the final accuracy
# use GridSearchCV to find the best parameters for the model
# use KerasClassifier to wrap the keras model into scikit learn
# use the best parameters to train the model
# save the best parameters for the model
# save the accuracy of the model

# define the parameters for the model
init = ['glorot_uniform']
neurons = [24, 12, 25, 49]
epochs = [1, 10, 50, 100, 500]
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.1, 0.5, 0.9, 0.99]
decay = [0.0, 0.1, 0.01, 0.001]
param_grid = dict(init = init, neurons = neurons, epochs = epochs, learn_rate = learn_rate, momentum = momentum, decay = decay)
SKfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = None)

# define the model
classifier = KerasClassifier(build_fn = build_classifier(init = 'glorot_uniform', neurons=17), init = 'uniform', neurons = 17, epochs = 100, batch_size = 10)

# define the grid search
grid = GridSearchCV(estimator = classifier, param_grid = param_grid, cv = SKfold)

# fit the grid search
grid_result = grid.fit(training_data, training_data_output)


# summarize the results
bp_dict = grid_result.get_params()
print(bp_dict)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))



INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmpev878ch4\assets


INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmpev878ch4\assets


INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmp7ogg9rga\assets


INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmp7ogg9rga\assets


INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmpjyjpw40u\assets


...

INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmpe_4t9m3s\assets


INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmplvezrehz\assets


INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmplvezrehz\assets


INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmp3q6yxvx3\assets


INFO:tensorflow:Assets written to: C:\Users\kwokt\AppData\Local\Temp\tmp3q6yxvx3\assets
